In [ ]:
import pandas as pd
import plotly.express as px

pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
pd.set_option('display.max_seq_items', None)

SEED = 2025

In [2]:

df_raw = pd.read_csv("../dados/temporarios/10-md_edb_cols_finais.csv").drop(columns=["lat", "lng"])


for col in df_raw.columns:
    try:
        df_raw[col] = df_raw[col].astype(int)
    except Exception as e:
        pass

print(df_raw.shape)
display(df_raw.info())


(37178, 24)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 37178 entries, 0 to 37177
Data columns (total 24 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   CO_ENTIDADE          37178 non-null  int64  
 1   QT_SALAS_UTILIZADAS  37178 non-null  int64  
 2   IN_EXAME_SELECAO     37178 non-null  int64  
 3   QT_MAT_INF           37178 non-null  int64  
 4   QT_MAT_FUND_AI       37178 non-null  int64  
 5   QT_MAT_FUND_AF       37178 non-null  int64  
 6   QT_MAT_MED           37178 non-null  int64  
 7   infraestrutura       37178 non-null  int64  
 8   estrutura_pobre      37178 non-null  int64  
 9   estrutura_basica     37178 non-null  int64  
 10  estrutura_padrao     37178 non-null  int64  
 11  estrutura_premium    37178 non-null  int64  
 12  qt_ativos_basico     37178 non-null  int64  
 13  qt_ativos_premium    37178 non-null  int64  
 14  pessoal_basico       37178 non-null  int64  
 15  pessoal_padrao       371

None

## Criando colunas melhores

## Correlação entre atributos

In [6]:

import plotly.express as px

corr = df_corr.drop(columns=["CO_ENTIDADE", "cliente"]).corr(numeric_only=True)

fig = px.imshow(
    corr,
    text_auto=False,
    aspect="auto",
    color_continuous_scale="Viridis",
    zmin=-1,
    zmax=1,
)
fig.update_layout(width=900, height=900)
fig.show()


## One-Class SVM

In [7]:
import numpy as np
import pandas as pd
from sklearn.svm import OneClassSVM
from sklearn.preprocessing import StandardScaler
from sklearn.impute import SimpleImputer
from sklearn.decomposition import PCA
from sklearn.pipeline import Pipeline
from sklearn.model_selection import ParameterGrid

def ocsvm_opt(df, cliente_col="cliente", id_col="CO_ENTIDADE", var_explicada=0.95):
    """
    Executa One-Class SVM otimizado para identificar observações semelhantes aos clientes.

    Entradas:
        df : DataFrame original
        cliente_col : nome da coluna com 1 (cliente conhecido) e NaN (desconhecido)
        id_col : coluna identificadora
        var_explicada : variância retida pelo PCA

    Saída:
        df_result : DataFrame contendo:
            - id
            - score do OC-SVM
            - rótulo binário
            - features originais
            - ordenado do mais provável ao menos provável
    """

    # 1) Separar clientes positivos conhecidos e desconhecidos
    df_pos = df[df[cliente_col] == 1].copy()
    df_unl = df[df[cliente_col].isna()].copy()

    # 2) Selecionar features numéricas (excluindo id e rótulo)
    features = [c for c in df.select_dtypes(include=[np.number]).columns 
                if c not in [cliente_col, id_col]]
    # Atenção: id_col pode não ser numérico; se for numérico, removemos explicitamente
    if id_col in features:
        features.remove(id_col)

    X_train = df_pos[features]
    X_test  = df_unl[features]

    # 3) Pipeline de pré-processamento (imputação + escala + PCA)
    pre = Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("pca", PCA(n_components=var_explicada))
    ])

    X_train_proc = pre.fit_transform(X_train)
    X_test_proc  = pre.transform(X_test)

    # 4) Grid de parâmetros (tamanho reduzido mas efetivo)
    param_grid = {
        "nu":    [0.01, 0.03, 0.05, 0.1],
        "gamma": ["scale", "auto", 0.01, 0.001]
    }

    best_score = -np.inf
    best_params = None
    best_model = None

    # 5) One-class cross-validation interna:
    #    Particionamos apenas entre positivos, usando um split simples.
    #    Para OC-SVM, usamos "quão grandes são os scores médios" como métrica.
    n = len(X_train_proc)
    idx = np.arange(n)
    np.random.shuffle(idx)
    mid = n // 2

    X_a = X_train_proc[idx[:mid]]
    X_b = X_train_proc[idx[mid:]]

    for params in ParameterGrid(param_grid):
        oc = OneClassSVM(kernel="rbf", **params)
        oc.fit(X_a)
        # score médio no outro bloco
        score = oc.decision_function(X_b).mean()
        if score > best_score:
            best_score = score
            best_params = params
            best_model = oc

        # inverter A/B (simetria)
        oc = OneClassSVM(kernel="rbf", **params)
        oc.fit(X_b)
        score = oc.decision_function(X_a).mean()
        if score > best_score:
            best_score = score
            best_params = params
            best_model = oc

    print(best_score)
    # 6) Re-treinar modelo final usando todos os positivos
    oc_final = OneClassSVM(kernel="rbf", **best_params)
    oc_final.fit(X_train_proc)

    # 7) Predição em todos
    df_proc  = pre.transform(df[features])

    scores = oc_final.decision_function(df_proc)
    bin_pred = oc_final.predict(df_proc)

    df_out = df.copy()
    df_out["ocsvm_score"] = scores
    df_out["ocsvm_binary"] = bin_pred

    # 8) Ordenar do mais provável para o menos provável
    df_out = df_out.sort_values("ocsvm_score", ascending=False)

    # 9) Retornar resultados completos
    return {
        "model": oc_final,
        "preprocess": pre,
        "best_params": best_params,
        "df_ranking": df_out
    }


b = ocsvm_opt(df_corr)



1.4902104362877522


In [8]:
df_b = b["df_ranking"]
df_b.head()

,CO_ENTIDADE,QT_SALAS_UTILIZADAS,IN_EXAME_SELECAO,QT_MAT_INF,QT_MAT_FUND_AI,QT_MAT_FUND_AF,QT_MAT_MED,infraestrutura,estrutura_pobre,estrutura_basica,estrutura_padrao,estrutura_premium,qt_ativos_basico,qt_ativos_premium,pessoal_basico,pessoal_padrao,pessoal_premium,qt_alunos,qt_professores,tipo_ocupacao,valor_venda,nota_objetiva,nota_redacao,cliente,alunos_p_professor,alunos_p_sala,ocsvm_score,ocsvm_binary
8002,27035018,24,0,35,204,253,180,3,0,2,3,3,39,62,18,6,2,672,46,3,106926,558.209043,764.680851,NaN,14.608696,28.000000,6.467067,1
14871,32039654,21,0,23,240,319,211,3,0,3,3,4,45,30,11,9,3,793,48,3,106926,556.534091,777.454545,NaN,16.520833,37.761905,6.455564,1
9673,29428408,25,0,83,200,314,274,3,0,4,4,4,109,18,30,14,5,871,65,3,106926,578.153636,805.090909,1.0,13.400000,34.840000,6.415556,1
4678,24003581,20,0,147,267,219,137,3,0,4,4,3,20,31,13,11,3,770,58,3,106926,584.535204,823.673469,NaN,13.275862,38.500000,6.283533,1
8848,29149860,30,0,73,292,319,214,3,0,4,3,1,40,40,30,16,5,898,68,3,106926,613.954167,805.000000,NaN,13.205882,29.933333,6.262135,1


In [9]:
import plotly.graph_objects as go

fig = go.Figure()

df_c = df_b#[df_b["cliente"] == 1]

fig.add_trace(go.Scatter(y=df_c["cliente"], mode='markers', name='cliente'))
fig.add_trace(go.Scatter(y=df_c["ocsvm_score"], mode='lines', name='prob_cliente'))
# fig.add_trace(go.Scatter(y=df_c["ocsvm_binary"], mode='markers', name='prob_binary'))

# fig.update_yaxes(range=[-0.1, 1.1])

fig.show()

In [10]:
break

SyntaxError: 'break' outside loop (668683560.py, line 1)

In [ ]:
df = df_raw.copy()

df_train = df[df['cliente'] == 1].copy()
df_test  = df[df['cliente'].isna()].copy()


features = [c for c in df.columns if c not in ['CO_ENTIDADE', 'cliente']]
X_train = df_train[features]
X_test  = df_test[features]


In [ ]:
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

pipe = Pipeline([
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler()),
    ('pca', PCA(n_components=0.95)),
])


X_train_proc = pipe.fit_transform(X_train)
X_test_proc  = pipe.transform(X_test)


In [ ]:
from sklearn.svm import OneClassSVM

ocsvm = OneClassSVM(kernel='rbf', gamma='scale', nu=0.1, gamma)
ocsvm.fit(X_train_proc)

In [ ]:

pred_binary = ocsvm.predict(X_test_proc)  # 1 cliente, -1 nao cliente (binario)
scores = ocsvm.decision_function(X_test_proc)

df_test['ocsvm_score'] = scores
df_test['ocsvm_binary'] = pred_binary

df_ranked = df_test.sort_values('ocsvm_score', ascending=False)


In [ ]:

df_simu = df.drop(columns=['CO_ENTIDADE', 'cliente'])
df_simu_b = pipe.transform(df_simu)
scores = ocsvm.decision_function(df_simu_b)

df_simu['ocsvm_score'] = scores
df_simu['CO_ENTIDADE'] = df['CO_ENTIDADE']
df_simu['cliente'] = df['cliente']

df_simu = df_simu.sort_values("ocsvm_score", ascending=False)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=df_simu["cliente"], mode='markers', name='cliente'))
fig.add_trace(go.Scatter(y=df_simu["ocsvm_score"], mode='lines', name='prob_cliente'))

# fig.update_yaxes(range=[-0.1, 1.1])

fig.show()

## CatBoost

Nao funcionou bem, overfit rapido e tbm sao poucos dados categoricos

In [ ]:
break

In [ ]:
df = df_raw.copy()

# TODO pensar em uma metrica para manter nota enem
df = df.drop(columns=["TP_OCUPACAO_PREDIO_ESCOLAR", "nota_objetiva", "nota_redacao"])


In [ ]:
cols_cat = ["IN_EXAME_SELECAO"]

In [ ]:
# ============================================================
# 2. Separação de labels e features
# cliente = 1 (positivo conhecido), NaN = unlabeled
# ============================================================
df2 = df.copy()

df2['label'] = df2['cliente'].fillna(0).astype(int)  # 1 = positivo, 0 = unlabeled

X = df2.drop(columns=["label", "cliente"]).astype(int)
y = df2['label']

In [ ]:
from catboost import CatBoostClassifier

# ============================================================
# 3. Treino do classificador PU (g(x) = P(s=1 | x))
# CatBoost lida com:
# - NaN
# - categóricas
# - mistura de tipos
# ============================================================

clf = CatBoostClassifier(
    iterations=4000,
    depth=6,
    learning_rate=0.04,
    eval_metric='AUC',
    loss_function='Logloss',
    verbose=200
)

# Índices das colunas categóricas no X
cat_idx = [X.columns.get_loc(c) for c in cols_cat]

clf.fit(
    X,
    y,
    cat_features=cat_idx
)

# Probabilidade de "ser rotulado como positivo" = g(x)
g = clf.predict_proba(X)[:, 1]

# ============================================================
# 4. Estimar c = P(s=1 | y=1)
# Média das previsões sobre os POSITIVOS conhecidos
# ============================================================
c = g[y == 1].mean()

# ============================================================
# 5. Probabilidade PU final
# P(y=1 | x) = g(x) / c
# ============================================================
p_true = (g / c).clip(0, 1)

df['prob_cliente'] = p_true

# ============================================================
# 6. Opcional: ordenação por probabilidade
# ============================================================
df_sorted = df.sort_values('prob_cliente', ascending=False)


In [ ]:
import plotly.graph_objects as go

fig = go.Figure()

fig.add_trace(go.Scatter(y=df_sorted["cliente"], mode='markers', name='cliente'))
fig.add_trace(go.Scatter(y=df_sorted["prob_cliente"], mode='lines', name='prob_cliente'))

fig.update_yaxes(range=[-0.1, 1.1])

fig.show()
